# Prueba del Agente con Control de Acceso

Este notebook prueba el agente `simple.py` con diferentes roles de usuario.

In [ ]:
import sys
sys.path.append('../src')

from simple import agent
from langchain_core.messages import HumanMessage

## Prueba 1: Usuario sin ID (Acceso Denegado)

In [ ]:
# Usuario sin proporcionar ID
result = agent.invoke({
    "user_role": "usuario",
    "user_id": None,
    "messages": [HumanMessage(content="Quiero ver mi información personal")]
})

print("Estado final:")
print(f"Acceso: {'✅ Concedido' if result['access_granted'] else '❌ Denegado'}")
print(f"\nRespuesta: {result['messages'][-1].content}")

## Prueba 2: Cliente con ID (Acceso Limitado)

In [ ]:
# Cliente con ID válido
result = agent.invoke({
    "user_role": "cliente",
    "user_id": 12345,
    "messages": [HumanMessage(content="Necesito actualizar mi dirección de correo electrónico")]
})

print("Estado final:")
print(f"Rol: {result['user_role']}")
print(f"ID: {result['user_id']}")
print(f"Acceso: {'✅ Concedido' if result['access_granted'] else '❌ Denegado'}")
print(f"\nRespuesta del agente:\n{result['messages'][-1].content}")

## Prueba 3: Empleado (Acceso Completo)

In [ ]:
# Empleado con acceso completo
result = agent.invoke({
    "user_role": "empleado",
    "user_id": 999,
    "messages": [HumanMessage(content="Dame un resumen de todos los usuarios registrados en el sistema")]
})

print("Estado final:")
print(f"Rol: {result['user_role']}")
print(f"Acceso: {'✅ Concedido' if result['access_granted'] else '❌ Denegado'}")
print(f"\nRespuesta del agente:\n{result['messages'][-1].content}")

## Prueba 4: Administrador (Acceso Completo)

In [ ]:
# Administrador con acceso completo
result = agent.invoke({
    "user_role": "administrador",
    "user_id": 1,
    "messages": [HumanMessage(content="Quiero modificar los datos del usuario con ID 12345")]
})

print("Estado final:")
print(f"Rol: {result['user_role']}")
print(f"Acceso: {'✅ Concedido' if result['access_granted'] else '❌ Denegado'}")
print(f"\nRespuesta del agente:\n{result['messages'][-1].content}")

## Prueba 5: Cliente intentando acceder a datos de otro usuario

In [ ]:
# Cliente intentando ver datos de otro usuario
result = agent.invoke({
    "user_role": "cliente",
    "user_id": 100,
    "messages": [HumanMessage(content="Muéstrame la información del usuario con ID 200")]
})

print("Estado final:")
print(f"Rol: {result['user_role']}")
print(f"ID del cliente: {result['user_id']}")
print(f"Acceso: {'✅ Concedido' if result['access_granted'] else '❌ Denegado'}")
print(f"\nRespuesta del agente:\n{result['messages'][-1].content}")
print("\n⚠️ El agente debería rechazar esta solicitud o advertir sobre permisos")

## Resumen del Sistema de Control de Acceso

### Roles y Permisos:

| Rol | Permisos |
|-----|----------|
| **Usuario** | Solo puede ver/modificar su propia información (requiere `user_id`) |
| **Cliente** | Solo puede ver/modificar su propia información (requiere `user_id`) |
| **Empleado** | Acceso completo a toda la información |
| **Administrador** | Acceso completo a toda la información |

### Flujo del Agente:

1. **check_access**: Verifica el rol y determina permisos
2. **Decisión condicional**: Si tiene acceso → continúa, si no → termina
3. **process_query**: Procesa la consulta con Groq LLM

### Base de Datos:
- PostgreSQL en Render
- Conexión configurada en `simple.py`
- Función `get_db_connection()` disponible para consultas